In [139]:
import torch
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# extract activation map for ISIC 2017 validation set

# compute culprit score from the activations, predictions, and gt

In [ ]:
from culprit import *
import json
config = json.load(open('config_skin_alexnet.json'))
os.environ["CUDA_VISIBLE_DEVICES"]= '1'
resume = 'skinmodel/checkpoint.pth'